# IBM Applied Data Science Capstone Project

## Week 5 Final Project

#### _Opening a New Shopping Mall in New Delhi, India_

* Build a dataframe of neighborhoods in New Delhi, India by Web Scraping the data from Wikipedia Page
* Get the geographical coordinates of the neighbor
* Obtain the venue data for neighborhoods from FourSquare API
* Explore, Analyze and Cluster the neighborhoods
* Select the best cluster to open a new Shopping Mall


__1. Import Libraries__

In [9]:
import numpy as np #library to handle data in a vectorized manner

import pandas as pd #library for data analysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json #library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values 
!pip install geocoder
import geocoder #to get coordinates 

import requests #library to handle requests
from bs4 import BeautifulSoup #library to parse HTML and XML Documents

from pandas.io.json import json_normalize #transform json file into a Pandas dataframe

#Matplotlib and associated plotting modules 
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip install folium
import folium #Map Rendering Library

print("Libraries Imported!")

     |████████████████████████████████| 102kB 5.1MB/s ta 0:00:01
Libraries Imported!


__2. Scrap Data from Wikipedia page into a DataFrame__

In [12]:
#Send the Get Request
data = requests.get("https://en.wikipedia.org/wiki/Category:Neighbourhoods_in_Delhi").text

In [13]:
#Parse data from the html to a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [14]:
#Create a list to store neighborhood data
neighborhoodList = []

In [16]:
# append the data into list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [78]:
#create a new DataFrame from the list

dl_df = pd.DataFrame({"Neighborhood": neighborhoodList})

dl_df.drop([0],axis = 0, inplace = True)

dl_df.reset_index(inplace=True)

dl_df.drop(['index'], axis =1, inplace= True)

dl_df

,Neighborhood
0,Ashok Nagar (Delhi)
1,Ashok Vihar
2,Ashram Chowk
3,Babarpur
4,"Badarpur, Delhi"
5,Bali Nagar
6,Bawana
7,Ber Sarai
8,Bhajanpura
9,Chanakyapuri


In [79]:
# print the number of rows of the dataframe
dl_df.shape

(138, 1)

__3. Get the geographical coordinates__

In [80]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{},Delhi, India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [81]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [get_latlng(neighborhood) for neighborhood in dl_df["Neighborhood"].tolist()]

In [82]:
coords

[[28.692230000000052, 77.30124000000006],
 [28.69037000000003, 77.17609000000004],
 [28.710597501792023, 77.32696517369723],
 [28.50738000000007, 77.30346000000003],
 [28.50738000000007, 77.30346000000003],
 [28.652234222889238, 77.12939224396462],
 [28.800590000000057, 77.03473000000008],
 [28.549540000000036, 77.18167000000005],
 [28.699880000000064, 77.25906000000003],
 [28.595060000000046, 77.18573000000004],
 [28.656270000000063, 77.23232000000007],
 [28.67671000000007, 77.21767000000006],
 [28.633940000000052, 77.21968000000004],
 [28.60761000000008, 77.08714000000003],
 [28.65457890544559, 77.23339989939495],
 [28.62832000000003, 77.24727000000007],
 [28.605920000000026, 77.08529000000004],
 [28.560590000000047, 77.24678000000006],
 [28.57298000000003, 77.23357000000004],
 [28.591510000000028, 77.12945000000008],
 [28.699110000000076, 77.19105000000008],
 [28.594857177133914, 77.16729160908383],
 [28.684700000000078, 77.32774000000006],
 [28.679040000000043, 77.31476000000004],


In [83]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
df_coords.head()

,Latitude,Longitude
0,28.692230,77.301240
1,28.690370,77.176090
2,28.710598,77.326965
3,28.507380,77.303460
4,28.507380,77.303460


In [84]:
# merge the coordinates into the original dataframe
dl_df['Latitude'] = df_coords['Latitude']
dl_df['Longitude'] = df_coords['Longitude']

In [85]:
# check the neighborhoods and the coordinates
print(dl_df.shape)

(138, 3)


In [86]:
# save the DataFrame as CSV file
dl_df.to_csv("dl_df.csv", index=False)

__4. Create a map of Delhi with Neighborhoods superimposed on top__

In [87]:
# get the coordinates of Delhi
address = 'Delhi, India'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Delhi, India is {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Delhi, India is 28.6517178, 77.2219388.


In [89]:
# create map of Delhi using latitude and longitude values
map_dl = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(dl_df['Latitude'], dl_df['Longitude'], dl_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_dl)  
    
map_dl

In [90]:
# save the map as HTML file
map_dl.save('map_dl.html')

__5. Use the Foursquare API to explore the neighborhoods__

In [91]:
# define Foursquare Credentials and Version
CLIENT_ID = 'ZPCI05YJDKCCUUOP2NZSZ2RAIMNGT22OQUQCPH3PKQH21WUW' # your Foursquare ID
CLIENT_SECRET = 'Q21GXCBJQTBYRMHPPRNSHPTXBLLGC00EFH5RWZBNB4PR2PRD' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ZPCI05YJDKCCUUOP2NZSZ2RAIMNGT22OQUQCPH3PKQH21WUW
CLIENT_SECRET:Q21GXCBJQTBYRMHPPRNSHPTXBLLGC00EFH5RWZBNB4PR2PRD


__Now, let's get the top 100 venues that are within a radius of 4000 meters.__

In [92]:
radius = 4000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(dl_df['Latitude'], dl_df['Longitude'], dl_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [93]:
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(9986, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Ashok Nagar (Delhi),28.69223,77.30124,Y.S.C - Yamuna Sports Complex,28.664500,77.312847,Athletics & Sports
1,Ashok Nagar (Delhi),28.69223,77.30124,Yamuna Sports Complex,28.664257,77.320179,Pool
2,Ashok Nagar (Delhi),28.69223,77.30124,METRO Cash & Carry,28.660832,77.297576,Big Box Store
3,Ashok Nagar (Delhi),28.69223,77.30124,The Leela Ambience Delhi Convention Hotel,28.662089,77.302429,Hotel
4,Ashok Nagar (Delhi),28.69223,77.30124,Barbeque Nation,28.665840,77.304094,BBQ Joint


__Let's check how many venues were returned for each neighborhood__

In [94]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Ashok Nagar (Delhi),21,21,21,21,21,21
Ashok Vihar,77,77,77,77,77,77
Ashram Chowk,5,5,5,5,5,5
Babarpur,26,26,26,26,26,26
"Badarpur, Delhi",26,26,26,26,26,26
Bali Nagar,76,76,76,76,76,76
Bawana,2,2,2,2,2,2
Ber Sarai,100,100,100,100,100,100
Bhajanpura,12,12,12,12,12,12


__Let's find out how many unique categories can be curated from all the returned venues__

In [95]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 186 uniques categories.


In [96]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Athletics & Sports', 'Pool', 'Big Box Store', 'Hotel',
       'BBQ Joint', 'Café', 'Pizza Place', 'Track', 'Shop & Service',
       'Park', 'Diner', 'Indian Restaurant', 'Light Rail Station',
       'Asian Restaurant', 'Shopping Mall', 'Fast Food Restaurant',
       'Playground', 'Donut Shop', 'Chinese Restaurant', 'Breakfast Spot',
       'Bakery', 'Sandwich Place', 'Miscellaneous Shop', 'Multiplex',
       'Snack Place', 'Coffee Shop', 'Vegetarian / Vegan Restaurant',
       'Department Store', 'Dumpling Restaurant',
       'South Indian Restaurant', 'Food Truck', 'Grocery Store',
       'Arts & Crafts Store', 'Restaurant', 'North Indian Restaurant',
       'Indian Sweet Shop', 'Train Station', 'Soccer Stadium',
       'Punjabi Restaurant', 'Historic Site', 'Stadium', 'Soccer Field',
       'American Restaurant', 'Market', 'Clothing Store', 'Hookah Bar',
       'Sculpture Garden', 'Sports Bar', 'Garden Center', 'Pub'],
      dtype=object)

In [97]:
# check if the results contain "Shopping Mall"
"Shopping Mall" in venues_df['VenueCategory'].unique()

True

__6. Analyze Each Neighborhood__

In [98]:
# one hot encoding
dl_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
dl_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [dl_onehot.columns[-1]] + list(dl_onehot.columns[:-1])
dl_onehot = dl_onehot[fixed_columns]

print(dl_onehot.shape)
dl_onehot.head()

(9986, 187)


,Neighborhoods,ATM,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bakery,Bank,Bar,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bistro,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Buffet,Burger Joint,Burmese Restaurant,Bus Station,Café,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Cricket Ground,Deli / Bodega,Department Store,Dessert Shop,Diner,Dog Run,Donut Shop,Dumpling Restaurant,Duty-free Shop,Electronics Store,English Restaurant,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Garden Center,Gastropub,Gift Shop,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Hindu Temple,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Karaoke Bar,Karnataka Restaurant,Korean Restaurant,Light Rail Station,Lounge,Luggage Store,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Mosque,Motel,Motorcycle Shop,Movie Theater,Moving Target,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Museum,Music Venue,Neighborhood,Nightclub,Nightlife Spot,North Indian Restaurant,Northeast Indian Restaurant,Other Nightlife,Palace,Paper / Office Supplies Store,Park,Performing Arts Venue,Pizza Place,Planetarium,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Public Art,Punjabi Restaurant,Racetrack,Recreation Center,Rental Car Location,Resort,Restaurant,Road,Salad Place,Sandwich Place,Scandinavian Restaurant,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South Indian Restaurant,Spa,Speakeasy,Spiritual Center,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Sushi Restaurant,Tapas Restaurant,Tea Room,Temple,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Tibetan Restaurant,Toll Booth,Tourist Information Center,Toy / Game Store,Track,Trail,Train Station,Travel Lounge,University,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Whisky Bar,Zoo
0,Ashok Nagar (Delhi),0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Ashok Nagar (Delhi),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Ashok Nagar (Delhi),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Ashok Nagar (Delhi),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

__Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category__

In [99]:
dl_grouped = dl_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(dl_grouped.shape)
dl_grouped

(138, 187)


,Neighborhoods,ATM,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bakery,Bank,Bar,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bistro,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Buffet,Burger Joint,Burmese Restaurant,Bus Station,Café,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Cricket Ground,Deli / Bodega,Department Store,Dessert Shop,Diner,Dog Run,Donut Shop,Dumpling Restaurant,Duty-free Shop,Electronics Store,English Restaurant,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Garden Center,Gastropub,Gift Shop,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Hindu Temple,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Karaoke Bar,Karnataka Restaurant,Korean Restaurant,Light Rail Station,Lounge,Luggage Store,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Mosque,Motel,Motorcycle Shop,Movie Theater,Moving Target,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Museum,Music Venue,Neighborhood,Nightclub,Nightlife Spot,North Indian Restaurant,Northeast Indian Restaurant,Other Nightlife,Palace,Paper / Office Supplies Store,Park,Performing Arts Venue,Pizza Place,Planetarium,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Public Art,Punjabi Restaurant,Racetrack,Recreation Center,Rental Car Location,Resort,Restaurant,Road,Salad Place,Sandwich Place,Scandinavian Restaurant,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South Indian Restaurant,Spa,Speakeasy,Spiritual Center,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Sushi Restaurant,Tapas Restaurant,Tea Room,Temple,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Tibetan Restaurant,Toll Booth,Tourist Information Center,Toy / Game Store,Track,Trail,Train Station,Travel Lounge,University,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Whisky Bar,Zoo
0,Ashok Nagar (Delhi),0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.047619,0.047619,0.000000,0.047619,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.047619,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.047619,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.047619,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.047619,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.142857,0.000000,0.000000,0.000000,0.095238,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.047619,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.142857,0.000000,0.047619,0.00,0.000000,0.000000,0.047619,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.047619,0.047619,0.000000,0.000000,0.000000,0.000000,0.

In [100]:
len(dl_grouped[dl_grouped["Shopping Mall"] > 0])

79

__Create a new DataFrame for Shopping Mall data only__

In [101]:
dl_mall = dl_grouped[["Neighborhoods","Shopping Mall"]]

In [102]:
dl_mall.head()

,Neighborhoods,Shopping Mall
0,Ashok Nagar (Delhi),0.047619
1,Ashok Vihar,0.038961
2,Ashram Chowk,0.000000
3,Babarpur,0.000000
4,"Badarpur, Delhi",0.000000


__7. Cluster Neighborhoods__ \
Run k-means to cluster the neighborhoods in Kuala Lumpur into 3 clusters.

In [104]:
# set number of clusters
kclusters = 3

dl_clustering = dl_mall.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(dl_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 0, 0, 0, 1, 0, 2, 0, 0], dtype=int32)

In [111]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
dl_merged = dl_mall.copy()

# add clustering labels
dl_merged["Cluster Labels"] = kmeans.labels_

In [112]:
dl_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
dl_merged.head()

,Neighborhood,Shopping Mall,Cluster Labels
0,Ashok Nagar (Delhi),0.047619,1
1,Ashok Vihar,0.038961,1
2,Ashram Chowk,0.000000,0
3,Babarpur,0.000000,0
4,"Badarpur, Delhi",0.000000,0


In [113]:
# merge dl_grouped with dl_data to add latitude/longitude for each neighborhood
dl_merged = dl_merged.join(dl_df.set_index("Neighborhood"), on="Neighborhood")

print(dl_merged.shape)
dl_merged.head()

(138, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Ashok Nagar (Delhi),0.047619,1,28.692230,77.301240
1,Ashok Vihar,0.038961,1,28.690370,77.176090
2,Ashram Chowk,0.000000,0,28.710598,77.326965
3,Babarpur,0.000000,0,28.507380,77.303460
4,"Badarpur, Delhi",0.000000,0,28.507380,77.303460


In [114]:
# sort the results by Cluster Labels
print(dl_merged.shape)
dl_merged.sort_values(["Cluster Labels"], inplace=True)
dl_merged

(138, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
137,Yamuna Vihar,0.000000,0,28.700590,77.272120
56,Laxmibai Nagar,0.000000,0,28.578150,77.206180
54,Lajpat Nagar,0.000000,0,28.570260,77.247000
53,"Krishna Nagar, Delhi",0.010000,0,28.563640,77.193670
52,Kotla Mubarakpur Complex,0.000000,0,28.574350,77.224190
105,Roop Nagar,0.000000,0,28.683720,77.197470
50,Kingsway Camp,0.000000,0,28.711690,77.201970
57,Lodhi Colony,0.000000,0,28.584760,77.225340
49,Khari Baoli,0.000000,0,28.657260,77.222840
107,Safdarjung (Delhi),0.000000,0,28.565830,77.199070


__Finally, let's visualize the resulting clusters__

In [115]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(dl_merged['Latitude'], dl_merged['Longitude'], dl_merged['Neighborhood'], dl_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [116]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

__8. Examine Clusters__

__Cluster 0__

In [117]:
dl_merged.loc[dl_merged['Cluster Labels'] == 0]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
137,Yamuna Vihar,0.000000,0,28.700590,77.272120
56,Laxmibai Nagar,0.000000,0,28.578150,77.206180
54,Lajpat Nagar,0.000000,0,28.570260,77.247000
53,"Krishna Nagar, Delhi",0.010000,0,28.563640,77.193670
52,Kotla Mubarakpur Complex,0.000000,0,28.574350,77.224190
105,Roop Nagar,0.000000,0,28.683720,77.197470
50,Kingsway Camp,0.000000,0,28.711690,77.201970
57,Lodhi Colony,0.000000,0,28.584760,77.225340
49,Khari Baoli,0.000000,0,28.657260,77.222840
107,Safdarjung (Delhi),0.000000,0,28.565830,77.199070


__Cluster 1__

In [119]:
dl_merged.loc[dl_merged['Cluster Labels'] == 1]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
131,Vasant Kunj,0.052632,1,28.531520,77.150200
134,Vivek Vihar subdivision,0.057971,1,28.646420,77.306150
115,Seemapuri subdivision,0.047619,1,28.687600,77.330620
94,Paschim Vihar,0.045455,1,28.669330,77.091730
96,Pitam Pura,0.060606,1,28.695900,77.137250
97,Punjabi Bagh,0.054348,1,28.666340,77.125000
116,Shahdara district,0.065574,1,28.653900,77.296410
104,"Rohini, Delhi",0.047619,1,28.733560,77.104010
124,Shivaji Place,0.037500,1,28.652650,77.121390
106,"Sadar Bazaar, Delhi",0.047619,1,28.590280,77.120140


__Cluster 2__

In [120]:
dl_merged.loc[dl_merged['Cluster Labels'] == 2]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
32,Greater Kailash,0.020000,2,28.548490,77.236670
27,Geetanjali Enclave,0.020000,2,28.528450,77.203540
24,"Dwarka, Delhi",0.028571,2,28.589950,77.040040
73,Munirka,0.030000,2,28.555040,77.171320
21,Dhaula Kuan,0.020000,2,28.594857,77.167292
125,South Extension,0.020000,2,28.562490,77.215170
126,South Patel Nagar,0.015152,2,28.645610,77.166820
95,Patel Nagar,0.013514,2,28.647830,77.164490
128,Tilak Nagar (Delhi),0.027778,2,28.640890,77.089220
19,Delhi Cantonment,0.035714,2,28.591510,77.129450


__Observations__

Most of the shopping malls are concentrated in the southern area of city, with the highest number in cluster 2 and moderate number in cluster 1. On the other hand, cluster 0 has very low number to totally no shopping mall in the neighborhoods. This represents a great opportunity and high potential areas to open new shopping malls as there is very little to no competition from existing malls. Meanwhile, shopping malls in cluster 2 are likely suffering from intense competition due to oversupply and high concentration of shopping malls. From another perspective, this also shows that the oversupply of shopping malls mostly happened in the southern area of the city, with the suburb area still have very few shopping malls. Therefore, this project recommends property developers to capitalize on these findings to open new shopping malls in neighborhoods in cluster 0 with little to no competition. Property developers with unique selling propositions to stand out from the competition can also open new shopping malls in neighborhoods in cluster 1 with moderate competition. Lastly, property developers are advised to avoid neighborhoods in cluster 2 which already have high concentration of shopping malls and suffering from intense competition.